<a href="https://colab.research.google.com/github/FabriceBeaumont/1-MA-INF_2314_IPSA_Repo/blob/main/Code/Exercise4/SolutionsOfSheet4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IPSA 2021 - Exercise 4 - Fourier Transforms and Gaussian filters

## 0 - Practical Advice

In [4]:
import imageio 
import numpy as np
import numpy.fft as fft
import numpy.random as rnd
import scipy.ndimage as img
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 6]
import timeit, functools

In [5]:
def imageScaleRange(f, vmin=0, vmax=255):
    """Stretches the intensity values of a given image array f 
    such that they cover the whole range from 0 to 255"""
    return np.interp(f, (f.min(), f.max()), (vmin, vmax))

def imageRead(imgname, pilmode ='L', arrtype=float):
    """
    Read an image file into a numpy array

    imgname: str
        name of image file to be read
    pilmode: str
        for luminance / intesity images use ’L’
        for RGB color images use ’RGB’
    
    arrtype: numpy dtype
        use np.float, np.uint8, ...
    """
    return imageio.imread(imgname, pilmode=pilmode).astype(arrtype)

def imageWrite(arrF, imgname, arrtype=np.uint8):
    """
    Write a numpy array as an image file 
    the file type is inferred from the suffix of parameter imgname, e.g. ’.png’
    arrF: array_like
        array to be written
    imgname: str
        name of image file to be written
    arrtype: numpy dtype
        use np.uint8, ...
    """
    imageio.imwrite(imgname, arrF.astype(arrtype))

In [6]:
def writeFigure(fig, fname):
    fmt = fname.split('.')[-1]
    fig.savefig(fname, facecolor=fig.get_facecolor(), edgecolor='w',
                papertype='letter', format=fmt, transparent=False,
                bbox_inches='tight', pad_inches=0)

def plotFcts(x, fs,
             labs=None, lfs='x-small',
             xtcs=None, ytcs=None,
             lims=None, fname=None, title=None):

    # Initialize figure and axes
    fig = plt.figure()
    axs = fig.add_subplot(211, facecolor='w')

    # Nicer way of showing coordinate axes
    for pos in ['left','bottom']:
        axs.spines[pos].set_position('zero')
    for pos in ['right','top']:
        axs.spines[pos].set_visible(False)
        
    # Plot the function
    for f in fs:
        axs.plot(x, f, '-', alpha=0.75)

    # Plot the legend
    if labs is not None:
        axs.legend(labs, loc="upper right", fontsize=lfs,
                   facecolor='#e0e0e0', edgecolor='k', fancybox=False)

    if xtcs is not None: axs.set_xticks(xtcs)
    if ytcs is not None: axs.set_yticks(ytcs)
    
    if lims is not None:
        axs.set_xlim(lims[0], lims[1])
        axs.set_ylim(lims[2], lims[3])
        
    if title:
        plt.title(title)
        
        
    # Show figure on screen or write it to disc
    plt.show() if fname is None else writeFigure(fig, fname)
    plt.close()

## 1 - Plotting boundaries of shape images

In the Data folder for this exercise, you will find the data file `hand.csv` which contains a data matrix $X\in\mathbb{R}^{2\times 537}$ whose columns $x_k = [x_k , y_k ]^\intercal$ represent 2D points. If you plot these data points using individual dots or a closed line.

![dot_and_line.png](attachment:dot_and_line.png)

To read these data into a numpy array `matX` representing matrix $X$, you may proceed as follows:

In [7]:
matX = np.loadtxt('hand.csv', delimiter=', ')

Now, center the data in $X$, i.e. determine the the mean $\hat{x}$ of the data points $x_k$ and transform them as follows:
$$  x_k  \leftarrow \quad x_k - \hat{x} $$


Create a dot plot and a line plot of the centered data (including coordinate axes).

## 2 - Interpolating boundaries of shape images

In this course, we work extensively with the python modules `numpy` / `scipy` because they provide numerous functionalities for numerical computing and number crunching. However, during the lectures, we also mentioned software for symbolic computation. Maybe you have come across such software before? Maybe you have heard of Wolfram Mathematica? For python, functionalities for symbolic computation are provided by the `sympy` module. To see what it buys you, execute the following snippet:

## 3 - Fourier Transforms of Gaussians and their Derivatives

### 3 (d) Using the inverse Fourier transform to compute a derivative